# plot

> A module to plot the results of the benchmark

In [ ]:
#| default_exp plot

In [ ]:
#| export
import plotly.graph_objects as go
from fasterbench.utils import *

In [ ]:
#| export
def create_radar_plot(benchmark_results):
    # Define metrics with icons, hover text format, and units
    metrics = {
        '💾': {  # Storage icon
            'value': benchmark_results['disk_size'] / 1e6,
            'hover_format': 'Model Size: {:.2f} MB',
            'unit': 'MB'
        },
        '🧮': {  # Calculator icon for parameters
            'value': parse_metric_value(benchmark_results['num_parameters']),
            'hover_format': 'Parameters: {:.2f}M',
            'unit': 'M'
        },
        '⏱️': {  # Clock icon for latency
            'value': benchmark_results['cpu_latency'],
            'hover_format': 'Latency: {:.2f} ms',
            'unit': 'ms'
        },
        '⚡': {  # Lightning bolt for MACs
            'value': parse_metric_value(benchmark_results['macs']),
            'hover_format': 'MACs: {:.2f}G',
            'unit': 'G'
        },
        '🔋': {  # Battery icon for energy
            'value': benchmark_results['avg_energy'] * 1e6,
            'hover_format': 'Energy: {:.3f} mWh',
            'unit': 'mWh'
        }
    }
    
    # Find min and max values for each metric
    reference_values = {
        '💾': {'min': 0, 'max': max(metrics['💾']['value'], 500)},    # Model size (MB)
        '🧮': {'min': 0, 'max': max(metrics['🧮']['value'], 50)},     # Parameters (M)
        '⏱️': {'min': 0, 'max': max(metrics['⏱️']['value'], 200)},    # Latency (ms)
        '⚡': {'min': 0, 'max': max(metrics['⚡']['value'], 5000)},      # MACs (G)
        '🔋': {'min': 0, 'max': max(metrics['🔋']['value'], 5)}       # Energy (mWh)
    }
    
    # Normalize values and create hover text
    normalized_values = []
    hover_texts = []
    labels = []
    
    for icon, metric in metrics.items():
        # Min-max normalization
        normalized_value = (metric['value'] - reference_values[icon]['min']) / \
                         (reference_values[icon]['max'] - reference_values[icon]['min'])
        normalized_values.append(normalized_value)
        
        # Create hover text with actual value
        hover_texts.append(metric['hover_format'].format(metric['value']))
        labels.append(icon)
    
    # Add first values again to close the polygon
    normalized_values.append(normalized_values[0])
    hover_texts.append(hover_texts[0])
    labels.append(labels[0])
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatterpolar(
        r=normalized_values,
        theta=labels,
        fill='toself',
        name='Model Metrics',
        hovertext=hover_texts,
        hoverinfo='text',
        line=dict(color='#FF8C00'),  # Bright orange color
        fillcolor='rgba(255, 140, 0, 0.3)'  # Semi-transparent orange
    ))
    
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 1],
                showticklabels=False,  # Hide radial axis labels
                gridcolor='rgba(128, 128, 128, 0.5)',  # Semi-transparent grey grid lines
                linecolor='rgba(128, 128, 128, 0.5)'   # Semi-transparent grey axis lines
            ),
            angularaxis=dict(
                tickfont=dict(size=24),  # Icon labels
                gridcolor='rgba(128, 128, 128, 0.5)'  # Semi-transparent grey grid lines
            ),
            bgcolor='rgba(0,0,0,0)'  # Transparent background
        ),
        showlegend=False,

        margin=dict(t=100, b=100, l=100, r=100),
        paper_bgcolor='rgba(0,0,0,0)',  # Transparent background
        plot_bgcolor='rgba(0,0,0,0)'    # Transparent background
    )
    
    return fig

In [ ]:
#| export
def create_tradeoff_plot(model_metrics_list, model_names, x_metric, y_metric, 
                         x_label=None, y_label=None, show_pareto=True,
                         higher_is_better_x=False, higher_is_better_y=True,
                         title=None, width=800, height=600):

    import plotly.graph_objects as go
    import numpy as np
    from plotly.subplots import make_subplots
    
    # Extract data points
    x_values = []
    y_values = []
    hover_texts = []
    
    for i, metrics in enumerate(model_metrics_list):
        try:
            # Handle string metric values with units
            x_val = parse_metric_value(metrics.get(x_metric, 0))
            y_val = parse_metric_value(metrics.get(y_metric, 0))
            
            x_values.append(x_val)
            y_values.append(y_val)
            
            # Create hover text with model name and both metrics
            hover_text = f"{model_names[i]}<br>{x_metric}: {x_val}<br>{y_metric}: {y_val}"
            hover_texts.append(hover_text)
        except (ValueError, KeyError):
            print(f"Warning: Could not extract {x_metric} or {y_metric} for model {model_names[i]}")
    
    # Create figure
    fig = make_subplots()
    
    # Add scatter plot for models
    fig.add_trace(go.Scatter(
        x=x_values,
        y=y_values,
        mode='markers+text',
        marker=dict(
            size=12,
            color='rgba(31, 119, 180, 0.8)',
            line=dict(width=1, color='black')
        ),
        text=model_names,
        textposition="top center",
        hovertext=hover_texts,
        hoverinfo='text',
        name='Models'
    ))
    
    # Compute and add Pareto frontier if requested
    if show_pareto and len(x_values) > 1:
        # Convert "higher is better" to "lower is better" for algorithm
        x_arr = np.array([-x if higher_is_better_x else x for x in x_values])
        y_arr = np.array([-y if higher_is_better_y else y for y in y_values])
        
        # Find Pareto-optimal points
        pareto_points = []
        pareto_indices = []
        
        for i in range(len(x_arr)):
            dominated = False
            for j in range(len(x_arr)):
                if i != j:
                    if x_arr[j] <= x_arr[i] and y_arr[j] <= y_arr[i] and (x_arr[j] < x_arr[i] or y_arr[j] < y_arr[i]):
                        dominated = True
                        break
            if not dominated:
                pareto_points.append((x_values[i], y_values[i]))
                pareto_indices.append(i)
        
        # Sort points for line drawing
        if higher_is_better_x:
            pareto_points.sort(key=lambda p: p[0])  # Sort by x ascending
        else:
            pareto_points.sort(key=lambda p: p[0], reverse=True)  # Sort by x descending
        
        # Extract sorted x and y values
        pareto_x = [p[0] for p in pareto_points]
        pareto_y = [p[1] for p in pareto_points]
        
        # Add line connecting Pareto-optimal points
        fig.add_trace(go.Scatter(
            x=pareto_x,
            y=pareto_y,
            mode='lines',
            line=dict(color='rgba(255, 0, 0, 0.7)', width=2, dash='dash'),
            name='Pareto Frontier'
        ))
        
        # Highlight Pareto-optimal points
        fig.add_trace(go.Scatter(
            x=[x_values[i] for i in pareto_indices],
            y=[y_values[i] for i in pareto_indices],
            mode='markers',
            marker=dict(
                size=16,
                color='rgba(255, 0, 0, 0.7)',
                symbol='circle-open',
                line=dict(width=3, color='red')
            ),
            hoverinfo='skip',
            showlegend=False
        ))
    
    # Set axis labels
    fig.update_layout(
        xaxis_title=x_label if x_label else x_metric,
        yaxis_title=y_label if y_label else y_metric,
        title=title if title else f"{y_metric} vs {x_metric} Trade-off",
        width=width,
        height=height,
        template="plotly_white",
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99
        )
    )
    
    # Add annotations for ideal direction
    annotations = []
    
    # X-axis direction arrow
    arrow_x = {"x": 0.95, "y": 0.02, "text": "→ Better", "showarrow": False, 
               "xref": "paper", "yref": "paper", "font": {"size": 12}}
    if not higher_is_better_x:
        arrow_x["x"] = 0.05
        arrow_x["text"] = "← Better"
    annotations.append(arrow_x)
    
    # Y-axis direction arrow
    arrow_y = {"x": 0.01, "y": 0.95, "text": "→ Better", "showarrow": False, 
               "xref": "paper", "yref": "paper", "textangle": -90, "font": {"size": 12}}
    if not higher_is_better_y:
        arrow_y["y"] = 0.05
        arrow_y["text"] = "↓ Better"
    annotations.append(arrow_y)
    
    fig.update_layout(annotations=annotations)
    
    return fig

In [ ]:
#| export
def plot_tradeoffs(model_list, dls, model_names=None):
    """Benchmark multiple models and create comparative plots"""
    if model_names is None:
        model_names = [f"Model {i+1}" for i in range(len(model_list))]
    
    # Collect metrics for all models
    metrics_list = []
    for model in model_list:
        dummy_input = torch.Tensor(next(iter(dls.valid))[0][0][None]).to('cpu')
        metrics = compute_model_metrics(model, dls, dummy_input)
        metrics_list.append(metrics)
    
    # Create tradeoff plots
    acc_lat_plot = create_tradeoff_plot(
        metrics_list,
        model_names,
        x_metric="latency", 
        y_metric="accuracy",
        x_label="Latency (ms/sample)",
        y_label="Accuracy (%)",
        higher_is_better_x=False,
        higher_is_better_y=True,
        title="Accuracy vs Latency Trade-off"
    )
    acc_lat_plot.show()
    
    size_acc_plot = create_tradeoff_plot(
        metrics_list, 
        model_names,
        x_metric="params", 
        y_metric="accuracy",
        x_label="Model Size (M params)",
        y_label="Accuracy (%)",
        higher_is_better_x=False,  # Smaller models are better
        higher_is_better_y=True    # Higher accuracy is better
    )
    size_acc_plot.show()